# Guía Práctica para la Gobernanza de la IA y el Combate al Sesgo Algorítmico 🛡️

En esta guía práctica, vamos a **probar** los conceptos clave que exploramos en clase sobre la **Gobernanza de la IA**, el insidioso **sesgo algorítmico** y la **importancia fundamental de los datos y su metadata**.

Prepárate para **ejecutar este código** y sumergirte en el proceso:
*   Verás cómo se **generan datos sintéticos** (¡incluso introduciremos un sesgo intencional para aprender de forma práctica!).
*   Descubrirás la vital importancia de **documentar todo con metadata**.

Entender y aplicar estos conceptos es **clave** para construir proyectos de IA que sean verdaderamente **justos, transparentes y responsables**.

**Hilo Conductor:** "Sesgos Algorítmicos: ¿Cómo los detectamos y combatimos desde la base? La Gobernanza de la IA empieza en los datos."

In [27]:
# Paso 1: Instalamos las librerías necesarias
# Si no las tenés instaladas en tu entorno de Colab, este comando las instala.

!pip install Faker pandas -q

print("Librerías instaladas correctamente. ¡Listos para empezar!")

Librerías instaladas correctamente. ¡Listos para empezar!


In [28]:
# Paso 2: Importamos lo que vamos a usar
# Faker: Para generar datos falsos (sintéticos) pero con aspecto real.
# pandas: Para manejar los datos en un formato de tabla (DataFrame).
# json: Para trabajar con la metadata en formato JSON, que es un estándar para intercambiar información.

from faker import Faker
import pandas as pd
import random
import json
import datetime

print("Librerías importadas.")

Librerías importadas.


In [29]:
# Paso 3: Inicializamos Faker
# Acá le decimos a Faker que genere datos con estilo argentino (es_AR).
# Esto es genial para que los ejemplos se sientan más cercanos.

fake = Faker('es_AR')

print(f"Faker inicializado con locale '{fake.locale}'.")

Faker inicializado con locale '<bound method BaseProvider.locale of <faker.providers.user_agent.Provider object at 0x7b3ffa7bf7d0>>'.


# Módulo 1: Generación de Datos Sintéticos con Sesgo Intencional

Acá vamos a crear un dataset sintético de "empleados".
¿Te acordás el caso de Amazon y el sesgo en la contratación?

Para que veas cómo se cuela un sesgo en los datos, vamos a introducir uno intencionalmente en el campo `genero_reportado`. Vamos a hacer que haya un 70% de hombres y un 30% de mujeres para esta demo.

*   Esto simula un "sesgo de proporción" o "sesgo histórico" que podríamos encontrar en datos reales si, por ejemplo, una empresa históricamente contrató más hombres para ciertos roles.

In [30]:
num_records = 50 # Un buen número para la demo

data_empleados = []
roles_disponibles = ['Desarrollador', 'Gerente de Proyectos', 'Analista de Datos', 'Diseñador UX', 'Especialista de RRHH', 'Contador', 'Asistente Administrativo']
departamentos_disponibles = ['IT', 'Ventas', 'Marketing', 'Finanzas', 'RRHH', 'Operaciones']

# Definimos una distribución sesgada para el género: 70% Masculino, 30% Femenino
# Esto se hace repitiendo 'Masculino' 7 veces y 'Femenino' 3 veces en la lista.
generos_con_sesgo = ['Masculino'] * 7 + ['Femenino'] * 3

print(f"Generando {num_records} registros de empleados. Atento al sesgo de género intencional.")

Generando 50 registros de empleados. Atento al sesgo de género intencional.


In [31]:
for i in range(num_records):
    # Elegimos el género según nuestra distribución sesgada
    genero_elegido = random.choice(generos_con_sesgo)

    # Asignamos el nombre según el género
    if genero_elegido == 'Masculino':
        nombre = fake.first_name_male()
    elif genero_elegido == 'Femenino':
        nombre = fake.first_name_female()
    else: # Por si agregamos más géneros en el futuro
        nombre = fake.first_name()

    data_empleados.append({
        'id_empleado': fake.unique.uuid4(), # ID único
        'nombre': nombre,
        'apellido': fake.last_name(),
        'genero_reportado': genero_elegido, # Campo con el sesgo
        'rol': random.choice(roles_disponibles),
        'salario': round(random.uniform(70000, 250000), 2), # Salarios aleatorios
        'departamento': random.choice(departamentos_disponibles),
        'fecha_contratacion': fake.date_this_decade() # Fecha dentro de los últimos 10 años
    })

# Convertimos la lista de diccionarios a un DataFrame de Pandas para un manejo más fácil
df_empleados = pd.DataFrame(data_empleados)

print("\n--- Dataset Sintético de Empleados Generado ---")


--- Dataset Sintético de Empleados Generado ---


In [32]:
# Mostramos las primeras filas para que vean cómo quedó
print(df_empleados.head())

print("\n--- Distribución del Género Reportado (¡Atento al Sesgo!) ---")
# Contamos cuántos registros hay de cada género para confirmar el sesgo
print(df_empleados['genero_reportado'].value_counts())

print("\nFijate que el campo 'genero_reportado' muestra un desbalance. Esto es un SESGO DE PROPORCIÓN, similar al que pudo haber existido en los datos de Amazon.")

                            id_empleado       nombre  apellido  \
0  0bf9aefe-5a22-4b41-abd9-424c272c9e6b       Thiago      Diaz   
1  96d11b61-6402-424e-8ef4-909424aa0d56  Juan Manuel      Ruiz   
2  bbd7ac48-a7a7-4fb1-a67c-43c9eedc9588     Benjamin     Ayala   
3  be305078-5166-4ae3-9cc3-c61e2e34fe9f     Bautista      Sosa   
4  107f8cfd-8ee5-4c9c-aaea-f8324815b20d     Isabella  Gonzalez   

  genero_reportado                   rol    salario departamento  \
0        Masculino  Especialista de RRHH  183282.07    Marketing   
1        Masculino              Contador  179546.51  Operaciones   
2        Masculino  Especialista de RRHH  127671.89    Marketing   
3        Masculino         Desarrollador  205761.86       Ventas   
4         Femenino  Especialista de RRHH   99415.62  Operaciones   

  fecha_contratacion  
0         2023-04-07  
1         2021-01-24  
2         2022-03-09  
3         2024-04-11  
4         2021-07-29  

--- Distribución del Género Reportado (¡Atento al Sesgo

# Módulo 2: Creación de Metadata para Nuestro Dataset

Ahora que tenemos nuestro dataset sintético, vamos a crear su metadata.
La metadata es el "ADN" del dataset. Nos dice todo sobre él: de dónde viene, qué contiene, ¡y crucialmente, qué sesgos conocidos tiene!
Sin metadata, es como tener una caja llena de cosas sin etiqueta: no sabés qué hay ni si es seguro usarlo.
Esta metadata es clave para la GOBERNANZA:

*   **Transparencia:** Cualquier persona puede entender el dataset.
*   **Trazabilidad:** Saber el origen y las transformaciones.
*   **Detección de Sesgos:** Documentar explícitamente los sesgos conocidos (¡como el que acabamos de crear!).
*   **Rendición de Cuentas:** Si hay un problema, podemos rastrear la fuente.

In [33]:
metadata_empleados = {
    "nombre_dataset": "empleados_sinteticos_demo_sesgo_genero",
    "descripcion": "Dataset de perfiles de empleados generado sintéticamente para fines educativos y demostración de sesgo algorítmico. NO CONTIENE DATOS REALES.",
    "fuente_datos": {
        "tipo": "Sintético",
        "herramienta": "Faker vX.Y.Z (reemplazar X.Y.Z por la versión usada)",
        "locale_faker": "es_AR",
        "metodo_generacion": "Generación aleatoria de campos con distribución de género ajustada intencionalmente (70% Masculino / 30% Femenino)."
    },
    "fecha_creacion": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "ultima_modificacion": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "version": "1.0",
    "licencia_uso": "Solo para fines educativos, investigación y desarrollo interno. NO para uso productivo o comercial. No usar para inferir patrones reales de empleo."
}

In [34]:
metadata_empleados["columnas"] = [
    {
        "nombre": "id_empleado",
        "tipo": "string (UUID)",
        "descripcion": "Identificador único de cada empleado. Generado aleatoriamente.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados['id_empleado'].iloc[0]
    },
    {
        "nombre": "nombre",
        "tipo": "string",
        "descripcion": "Nombre de pila del empleado. Generado aleatoriamente según el género asignado.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados['nombre'].iloc[0]
    },
    {
        "nombre": "apellido",
        "tipo": "string",
        "descripcion": "Apellido del empleado. Generado aleatoriamente.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados['apellido'].iloc[0]
    },
    {
        "nombre": "genero_reportado",
        "tipo": "string",
        "descripcion": "Género autodeclarado del empleado. Campo con sesgo intencional para demostración.",
        "sensibilidad": "Media (si fuera real, sería un dato personal sensible)",
        "procesamiento": "Seleccionado aleatoriamente de ['Masculino', 'Femenino'] con probabilidad de 70% y 30% respectivamente.",
        "distribucion_observada": df_empleados['genero_reportado'].value_counts().to_dict(),
        "advertencia_sesgo_campo": "Este campo introduce un sesgo de género en el dataset. Si se usa para entrenamiento de modelos, el modelo podría aprender a favorecer implícitamente al grupo mayoritario ('Masculino')."
    }
]

In [35]:
# Continuamos con el resto de columnas
metadata_empleados["columnas"].extend([
    {
        "nombre": "rol",
        "tipo": "string",
        "descripcion": "Rol laboral del empleado en la empresa.",
        "sensibilidad": "Baja",
        "ejemplo_valor": df_empleados['rol'].iloc[0]
    },
    {
        "nombre": "salario",
        "tipo": "float",
        "descripcion": "Salario anual del empleado en moneda local.",
        "sensibilidad": "Media (si fuera real, dato personal)",
        "ejemplo_valor": df_empleados['salario'].iloc[0]
    },
    {
        "nombre": "departamento",
        "tipo": "string",
        "descripcion": "Departamento al que pertenece el empleado.",
        "sensibilidad": "Baja",
        "ejemplo_valor": df_empleados['departamento'].iloc[0]
    },
    {
        "nombre": "fecha_contratacion",
        "tipo": "date",
        "descripcion": "Fecha en que el empleado fue contratado.",
        "sensibilidad": "Baja",
        "ejemplo_valor": str(df_empleados['fecha_contratacion'].iloc[0])
    }
])

In [36]:
# Agregamos la información crítica sobre sesgos conocidos
metadata_empleados["sesgos_conocidos_dataset"] = [
    {
        "tipo_sesgo": "Sesgo de Proporción / Desbalance de Clase",
        "campo_afectado": "genero_reportado",
        "descripcion": "La distribución de género en este dataset está *intencionalmente desbalanceada* (aproximadamente 70% hombres, 30% mujeres) para fines de demostración. Esto simula cómo un sesgo histórico o de muestreo en datos reales podría influir en un modelo.",
        "impacto_potencial": "Un modelo de aprendizaje automático (ej. para contratación, desempeño o promoción) entrenado con este dataset podría desarrollar sesgos de predicción o clasificación que favorezcan al grupo mayoritario ('Masculino'), perpetuando la discriminación."
    }
]

metadata_empleados["acciones_mitigacion_sugeridas"] = "Si este fuera un dataset real y se fuera a usar para un modelo de impacto, se requeriría aplicar técnicas de balanceo de clases (oversampling, undersampling), ajustar pesos en el modelo o usar algoritmos conscientes de equidad (como los explorados por IBM AI Fairness 360 o Fairlearn de Microsoft) antes de entrenar un modelo."

metadata_empleados["limitaciones_uso"] = "Este dataset *no* es representativo de la población laboral real y debe ser usado con extrema precaución. Su propósito es *demostrar* conceptos de sesgo y metadata, no para inferir patrones reales o para uso en sistemas productivos sin una validación exhaustiva."

In [37]:
# Imprimimos la metadata en un formato legible (JSON indentado)
print("\n--- Metadata Generada para el Dataset Sintético ---")
print(json.dumps(metadata_empleados, indent=4, ensure_ascii=False))

print("\n--- ¡Fijate bien en la sección 'sesgos_conocidos_dataset' de la metadata! ---")
print("Ahí es donde documentamos el sesgo que introdujimos. Esto es CRÍTICO para la transparencia.")


--- Metadata Generada para el Dataset Sintético ---
{
    "nombre_dataset": "empleados_sinteticos_demo_sesgo_genero",
    "descripcion": "Dataset de perfiles de empleados generado sintéticamente para fines educativos y demostración de sesgo algorítmico. NO CONTIENE DATOS REALES.",
    "fuente_datos": {
        "tipo": "Sintético",
        "herramienta": "Faker vX.Y.Z (reemplazar X.Y.Z por la versión usada)",
        "locale_faker": "es_AR",
        "metodo_generacion": "Generación aleatoria de campos con distribución de género ajustada intencionalmente (70% Masculino / 30% Femenino)."
    },
    "fecha_creacion": "2025-06-18 00:24:52",
    "ultima_modificacion": "2025-06-18 00:24:52",
    "version": "1.0",
    "licencia_uso": "Solo para fines educativos, investigación y desarrollo interno. NO para uso productivo o comercial. No usar para inferir patrones reales de empleo.",
    "columnas": [
        {
            "nombre": "id_empleado",
            "tipo": "string (UUID)",
          

# Módulo 3: Conexión con la Gobernanza y el Combate al Sesgo

¿Cómo se relaciona todo esto con la Gobernanza de la IA y el caso Amazon?

1.  **El Problema de Amazon:**
    Si Amazon hubiera tenido una Gobernanza de Datos robusta, habrían exigido metadata detallada para sus datos históricos de contratación. Esa metadata, como la que acabamos de crear, habría revelado el desbalance de género en sus datos.

2.  **La Metadata como "Alerta Temprana":**
    Al ver esa "advertencia de sesgo" en la metadata, el equipo habría podido:
    *   Auditar los datos con herramientas especializadas (¡como las que vimos de IBM!).
    *   Remediar el dataset (ej. balancearlo, si fuera posible, o ajustar técnicas de modelado).
    *   Aplicar algoritmos conscientes de equidad.

3.  **La Gobernanza es Prevención:**
    Esto demuestra que la Gobernanza de la IA, a través de prácticas como la creación rigurosa de metadata y la trazabilidad, es una herramienta PREVENTIVA. Te permite "diagnosticar" problemas en tus datos antes de que se traduzcan en sesgos en tu modelo y causen daños en el mundo real.

4.  **Tu Rol Fundamental:**
    Como futuro profesional de la IA, tenés la capacidad de influir directamente. Cada decisión que tomes, desde cómo elegís tus datos, cómo los generás (si son sintéticos) y cómo documentás esa información con metadata, tiene un impacto ético.

#PRACTICA


## Ejercicios Adicionales para Explorar

*   Cambiá la distribución de género para que sea 50/50 y observá cómo cambia la metadata.
*   Agregá otro tipo de sesgo, por ejemplo en los salarios según el género.
*   Experimentá con otros campos como edad, experiencia laboral, etc.
*   Investigá herramientas de detección de sesgo como IBM AI Fairness 360 o Fairlearn de Microsoft.
*   Creá tu propia metadata para un dataset que uses en otro proyecto.

In [38]:
# Paso 1: Instalamos las librerías necesarias
# Si no las tenés instaladas en tu entorno de Colab, este comando las instala.

!pip install Faker pandas -q

print("Librerías instaladas correctamente. ¡Listos para empezar!")

Librerías instaladas correctamente. ¡Listos para empezar!


In [39]:
# Paso 2: Importamos lo que vamos a usar
# Faker: Para generar datos falsos (sintéticos) pero con aspecto real.
# pandas: Para manejar los datos en un formato de tabla (DataFrame).
# json: Para trabajar con la metadata en formato JSON, que es un estándar para intercambiar información.

from faker import Faker
import pandas as pd
import random
import json
import datetime

print("Librerías importadas.")

Librerías importadas.


In [40]:
# Paso 3: Inicializamos Faker
# Acá le decimos a Faker que genere datos con estilo argentino (es_AR).
# Esto es genial para que los ejemplos se sientan más cercanos.

fake2 = Faker('es_AR')

print(f"Faker inicializado con locale '{fake2.locale}'.")

Faker inicializado con locale '<bound method BaseProvider.locale of <faker.providers.user_agent.Provider object at 0x7b3ff9b29850>>'.


In [41]:
num_records2 = 50 # Un buen número para la demo

data_empleados2 = []
roles_disponibles2 = ['Desarrollador', 'Gerente de Proyectos', 'Analista de Datos', 'Diseñador UX', 'Especialista de RRHH', 'Contador', 'Asistente Administrativo']
departamentos_disponibles2 = ['IT', 'Ventas', 'Marketing', 'Finanzas', 'RRHH', 'Operaciones']



In [42]:
# Definimos una distribución sesgada para el género: 50% Masculino, 50% Femenino
# Esto se hace repitiendo 'Masculino' 7 veces y 'Femenino' 3 veces en la lista.
generos_con_sesgo2 = ['Masculino'] * 5 + ['Femenino'] * 5

print(f"Generando {num_records2} registros de empleados. Atento al sesgo de género intencional.")

Generando 50 registros de empleados. Atento al sesgo de género intencional.


In [43]:
for i in range(num_records2):
    # Elegimos el género según nuestra distribución sesgada
    genero_elegido2 = random.choice(generos_con_sesgo2)

    # Asignamos el nombre según el género
    if genero_elegido2 == 'Masculino':
        nombre2 = fake2.first_name_male()
    elif genero_elegido2 == 'Femenino':
        nombre2 = fake2.first_name_female()
    else: # Por si agregamos más géneros en el futuro
        nombre2 = fake2.first_name()

    data_empleados2.append({
        'id_empleado': fake.unique.uuid4(), # ID único
        'nombre': nombre2,
        'apellido': fake2.last_name(),
        'genero_reportado': genero_elegido2, # Campo con el sesgo
        'rol': random.choice(roles_disponibles2),
        'salario': round(random.uniform(70000, 250000), 2), # Salarios aleatorios
        'departamento': random.choice(departamentos_disponibles2),
        'fecha_contratacion': fake2.date_this_decade() # Fecha dentro de los últimos 10 años
    })

# Convertimos la lista de diccionarios a un DataFrame de Pandas para un manejo más fácil
df_empleados2 = pd.DataFrame(data_empleados2)

print("\n--- Dataset Sintético de Empleados Generado ---")


--- Dataset Sintético de Empleados Generado ---


In [44]:
# Mostramos las primeras filas para que vean cómo quedó
#print(df_empleados2.head())

print("\n--- Distribución del Género Reportado (¡Atento al Sesgo!) ---")
# Contamos cuántos registros hay de cada género para confirmar el sesgo
print(df_empleados2['genero_reportado'].value_counts())


--- Distribución del Género Reportado (¡Atento al Sesgo!) ---
genero_reportado
Masculino    25
Femenino     25
Name: count, dtype: int64


In [45]:
metadata_empleados2 = {
    "nombre_dataset": "empleados_sinteticos_demo_sesgo_genero",
    "descripcion": "Dataset de perfiles de empleados generado sintéticamente para fines educativos y demostración de sesgo algorítmico. NO CONTIENE DATOS REALES.",
    "fuente_datos": {
        "tipo": "Sintético",
        "herramienta": "Faker vX.Y.Z (reemplazar X.Y.Z por la versión usada)",
        "locale_faker": "es_AR",
        "metodo_generacion": "Generación aleatoria de campos con distribución de género ajustada intencionalmente (70% Masculino / 30% Femenino)."
    },
    "fecha_creacion": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "ultima_modificacion": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "version": "1.0",
    "licencia_uso": "Solo para fines educativos, investigación y desarrollo interno. NO para uso productivo o comercial. No usar para inferir patrones reales de empleo."
}

In [46]:
metadata_empleados2["columnas"] = [
    {
        "nombre": "id_empleado",
        "tipo": "string (UUID)",
        "descripcion": "Identificador único de cada empleado. Generado aleatoriamente.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados2['id_empleado'].iloc[0]
    },
    {
        "nombre": "nombre",
        "tipo": "string",
        "descripcion": "Nombre de pila del empleado. Generado aleatoriamente según el género asignado.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados2['nombre'].iloc[0]
    },
    {
        "nombre": "apellido",
        "tipo": "string",
        "descripcion": "Apellido del empleado. Generado aleatoriamente.",
        "sensibilidad": "Baja (sintético, no real)",
        "ejemplo_valor": df_empleados2['apellido'].iloc[0]
    },
    {
        "nombre": "genero_reportado",
        "tipo": "string",
        "descripcion": "Género autodeclarado del empleado. Campo con sesgo intencional para demostración.",
        "sensibilidad": "Media (si fuera real, sería un dato personal sensible)",
        "procesamiento": "Seleccionado aleatoriamente de ['Masculino', 'Femenino'] con probabilidad de 70% y 30% respectivamente.",
        "distribucion_observada": df_empleados2['genero_reportado'].value_counts().to_dict(),
        "advertencia_sesgo_campo": "Este campo introduce un sesgo de género en el dataset. Si se usa para entrenamiento de modelos, el modelo podría aprender a favorecer implícitamente al grupo mayoritario ('Masculino')."
    }
]

In [47]:
# Continuamos con el resto de columnas
metadata_empleados2["columnas"].extend([
    {
        "nombre": "rol",
        "tipo": "string",
        "descripcion": "Rol laboral del empleado en la empresa.",
        "sensibilidad": "Baja",
        "ejemplo_valor": df_empleados2['rol'].iloc[0]
    },
    {
        "nombre": "salario",
        "tipo": "float",
        "descripcion": "Salario anual del empleado en moneda local.",
        "sensibilidad": "Media (si fuera real, dato personal)",
        "ejemplo_valor": df_empleados2['salario'].iloc[0]
    },
    {
        "nombre": "departamento",
        "tipo": "string",
        "descripcion": "Departamento al que pertenece el empleado.",
        "sensibilidad": "Baja",
        "ejemplo_valor": df_empleados2['departamento'].iloc[0]
    },
    {
        "nombre": "fecha_contratacion",
        "tipo": "date",
        "descripcion": "Fecha en que el empleado fue contratado.",
        "sensibilidad": "Baja",
        "ejemplo_valor": str(df_empleados2['fecha_contratacion'].iloc[0])
    }
])

In [48]:
# Agregamos la información crítica sobre sesgos conocidos
metadata_empleados2["sesgos_conocidos_dataset"] = [
    {
        "tipo_sesgo": "Sesgo de Proporción / Desbalance de Clase",
        "campo_afectado": "genero_reportado",
        "descripcion": "La distribución de género en este dataset está *intencionalmente desbalanceada* (aproximadamente 70% hombres, 30% mujeres) para fines de demostración. Esto simula cómo un sesgo histórico o de muestreo en datos reales podría influir en un modelo.",
        "impacto_potencial": "Un modelo de aprendizaje automático (ej. para contratación, desempeño o promoción) entrenado con este dataset podría desarrollar sesgos de predicción o clasificación que favorezcan al grupo mayoritario ('Masculino'), perpetuando la discriminación."
    }
]

metadata_empleados2["acciones_mitigacion_sugeridas"] = "Si este fuera un dataset real y se fuera a usar para un modelo de impacto, se requeriría aplicar técnicas de balanceo de clases (oversampling, undersampling), ajustar pesos en el modelo o usar algoritmos conscientes de equidad (como los explorados por IBM AI Fairness 360 o Fairlearn de Microsoft) antes de entrenar un modelo."

metadata_empleados2["limitaciones_uso"] = "Este dataset *no* es representativo de la población laboral real y debe ser usado con extrema precaución. Su propósito es *demostrar* conceptos de sesgo y metadata, no para inferir patrones reales o para uso en sistemas productivos sin una validación exhaustiva."

In [49]:
# Imprimimos la metadata en un formato legible (JSON indentado)
print("\n--- Metadata Generada para el Dataset Sintético ---")
print(json.dumps(metadata_empleados2, indent=4, ensure_ascii=False))

print("\n--- ¡Fijate bien en la sección 'sesgos_conocidos_dataset' de la metadata! ---")
print("Ahí es donde documentamos el sesgo que introdujimos. Esto es CRÍTICO para la transparencia.")


--- Metadata Generada para el Dataset Sintético ---
{
    "nombre_dataset": "empleados_sinteticos_demo_sesgo_genero",
    "descripcion": "Dataset de perfiles de empleados generado sintéticamente para fines educativos y demostración de sesgo algorítmico. NO CONTIENE DATOS REALES.",
    "fuente_datos": {
        "tipo": "Sintético",
        "herramienta": "Faker vX.Y.Z (reemplazar X.Y.Z por la versión usada)",
        "locale_faker": "es_AR",
        "metodo_generacion": "Generación aleatoria de campos con distribución de género ajustada intencionalmente (70% Masculino / 30% Femenino)."
    },
    "fecha_creacion": "2025-06-18 00:24:58",
    "ultima_modificacion": "2025-06-18 00:24:58",
    "version": "1.0",
    "licencia_uso": "Solo para fines educativos, investigación y desarrollo interno. NO para uso productivo o comercial. No usar para inferir patrones reales de empleo.",
    "columnas": [
        {
            "nombre": "id_empleado",
            "tipo": "string (UUID)",
          